# 18S Model Training and Evaluation

**Objective:** To build, train, and evaluate a deep learning classifier for the 18S rRNA gene (Eukaryotes) using the pre-processed data.

**Methodology:**
1. Load the 18S-specific training/testing data and encoders from disk.
2. Define the neural network architecture.
3. Train the model on the training data using the GPU.
4. Save, reload, and evaluate the final model's accuracy on the unseen test set.

In [1]:
import numpy as np
import tensorflow as tf
from scipy.sparse import load_npz
import pickle
from pathlib import Path
import sys

# Set up project path
project_root = Path.cwd().parent

# --- Verification Step: Check for GPU ---
print("--- TensorFlow Setup ---")
print(f"TensorFlow Version: {tf.__version__}")
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU detected: {gpu_devices[0]}")
else:
    print("WARNING: No GPU detected. TensorFlow will run on CPU.")
print("-" * 26)

--- TensorFlow Setup ---
TensorFlow Version: 2.10.1
GPU detected: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
--------------------------
